In [ ]:
{
  "cells": [
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "view-in-github",
        "colab_type": "text"
      },
      "source": [
        "<a href=\"https://colab.research.google.com/github/market-research-v1/notebooks/research/backtesting.ipynb\" target=\"_parent\"><img src=\"https://colab.research.google.com/assets/colab-badge.svg\" alt=\"Open In Colab\"/></a>"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "title"
      },
      "source": [
        "# Basic Backtesting Studies - Indian Stock Market Research System v1.0\n",
        "## Market Research System - Research Notebooks\n",
        "\n",
        "**Version:** 1.0 (2022)  \n",
        "**Focus:** Indian Stock Market Analysis  \n",
        "**Objective:** Comprehensive backtesting framework for trading strategies\n",
        "\n",
        "---\n",
        "\n",
        "### 📊 Overview\n",
        "This notebook implements backtesting studies for various trading strategies on Indian stocks including:\n",
        "- Technical indicator-based strategies\n",
        "- Moving average crossover systems\n",
        "- Mean reversion strategies\n",
        "- Momentum-based approaches\n",
        "- Multi-timeframe analysis\n",
        "\n",
        "### 🎯 Target Stocks\n",
        "- **Nifty 50 constituents**\n",
        "- **Bank Nifty stocks**\n",
        "- **Large-cap Indian equities**\n",
        "- **Sector-specific analysis**"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "id": "install_dependencies"
      },
      "outputs": [],
      "source": [
        "# Install required packages for Indian stock market analysis\n",
        "!pip install yfinance pandas numpy matplotlib seaborn plotly\n",
        "!pip install ta-lib-binary talib scipy scikit-learn\n",
        "!pip install nsepy requests beautifulsoup4\n",
        "!pip install pyfolio empyrical quantstats\n",
        "\n",
        "# Import libraries\n",
        "import yfinance as yf\n",
        "import pandas as pd\n",
        "import numpy as np\n",
        "import matplotlib.pyplot as plt\n",
        "import seaborn as sns\n",
        "import plotly.graph_objects as go\n",
        "import plotly.express as px\n",
        "from plotly.subplots import make_subplots\n",
        "import warnings\n",
        "warnings.filterwarnings('ignore')\n",
        "\n",
        "# Set display options\n",
        "pd.set_option('display.max_columns', None)\n",
        "pd.set_option('display.width', None)\n",
        "\n",
        "# Plotting style\n",
        "plt.style.use('seaborn-v0_8')\n",
        "sns.set_palette(\"husl\")\n",
        "\n",
        "print(\"✅ Dependencies installed and imported successfully!\")\n",
        "print(\"🇮🇳 Ready for Indian Stock Market Backtesting Analysis\")"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "data_setup"
      },
      "source": [
        "## 📈 Data Setup - Indian Stock Market\n",
        "\n",
        "### Major Indian Stock Symbols\n",
        "We'll focus on liquid, high-volume Indian stocks for reliable backtesting results."
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "id": "stock_symbols"
      },
      "outputs": [],
      "source": [
        "# Indian Stock Symbols (NSE format for yfinance)\n",
        "INDIAN_STOCKS = {\n",
        "    # Banking Sector\n",
        "    'HDFCBANK.NS': 'HDFC Bank',\n",
        "    'ICICIBANK.NS': 'ICICI Bank', \n",
        "    'SBIN.NS': 'State Bank of India',\n",
        "    'KOTAKBANK.NS': 'Kotak Mahindra Bank',\n",
        "    'AXISBANK.NS': 'Axis Bank',\n",
        "    \n",
        "    # IT Sector\n",
        "    'TCS.NS': 'Tata Consultancy Services',\n",
        "    'INFY.NS': 'Infosys',\n",
        "    'WIPRO.NS': 'Wipro',\n",
        "    'HCLTECH.NS': 'HCL Technologies',\n",
        "    'TECHM.NS': 'Tech Mahindra',\n",
        "    \n",
        "    # Large Cap Stocks\n",
        "    'RELIANCE.NS': 'Reliance Industries',\n",
        "    'ITC.NS': 'ITC Limited',\n",
        "    'HINDUNILVR.NS': 'Hindustan Unilever',\n",
        "    'LT.NS': 'Larsen & Toubro',\n",
        "    'BAJFINANCE.NS': 'Bajaj Finance',\n",
        "    \n",
        "    # Auto Sector\n",
        "    'MARUTI.NS': 'Maruti Suzuki',\n",
        "    'M&M.NS': 'Mahindra & Mahindra',\n",
        "    'TATAMOTORS.NS': 'Tata Motors',\n",
        "    \n",
        "    # Pharma Sector\n",
        "    'SUNPHARMA.NS': 'Sun Pharmaceutical',\n",
        "    'DRREDDY.NS': 'Dr. Reddys Laboratories'\n",
        "}\n",
        "\n",
        "# Market Indices\n",
        "INDIAN_INDICES = {\n",
        "    '^NSEI': 'Nifty 50',\n",
        "    '^NSEBANK': 'Bank Nifty',\n",
        "    '^CNXIT': 'Nifty IT'\n",
        "}\n",
        "\n",
        "print(f\"📊 Total Stocks for Analysis: {len(INDIAN_STOCKS)}\")\n",
        "print(f\"📈 Market Indices: {len(INDIAN_INDICES)}\")\n",
        "print(\"\\n🏦 Banking Stocks:\", [name for symbol, name in INDIAN_STOCKS.items() if 'Bank' in name])\n",
        "print(\"💻 IT Stocks:\", [name for symbol, name in INDIAN_STOCKS.items() if symbol in ['TCS.NS', 'INFY.NS', 'WIPRO.NS', 'HCLTECH.NS', 'TECHM.NS']])"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "id": "data_fetcher"
      },
      "outputs": [],
      "source": [
        "class IndianStockDataFetcher:\n",
        "    \"\"\"Data fetcher specifically designed for Indian stock market analysis\"\"\"\n",
        "    \n",
        "    def __init__(self, start_date='2022-01-01', end_date='2022-12-31'):\n",
        "        self.start_date = start_date\n",
        "        self.end_date = end_date\n",
        "        self.data_cache = {}\n",
        "        \n",
        "    def fetch_stock_data(self, symbol, period='1y'):\n",
        "        \"\"\"Fetch stock data with error handling\"\"\"\n",
        "        try:\n",
        "            if symbol in self.data_cache:\n",
        "                return self.data_cache[symbol]\n",
        "                \n",
        "            stock = yf.Ticker(symbol)\n",
        "            data = stock.history(start=self.start_date, end=self.end_date)\n",
        "            \n",
        "            if data.empty:\n",
        "                print(f\"⚠️ No data found for {symbol}\")\n",
        "                return None\n",
        "                \n",
        "            # Clean data\n",
        "            data = data.dropna()\n",
        "            data.columns = data.columns.str.lower()\n",
        "            \n",
        "            # Cache the data\n",
        "            self.data_cache[symbol] = data\n",
        "            \n",
        "            return data\n",
        "            \n",
        "        except Exception as e:\n",
        "            print(f\"❌ Error fetching data for {symbol}: {str(e)}\")\n",
        "            return None\n",
        "    \n",
        "    def fetch_multiple_stocks(self, symbols_dict):\n",
        "        \"\"\"Fetch data for multiple stocks\"\"\"\n",
        "        stock_data = {}\n",
        "        failed_stocks = []\n",
        "        \n",
        "        print(f\"📥 Fetching data for {len(symbols_dict)} stocks...\")\n",
        "        \n",
        "        for symbol, name in symbols_dict.items():\n",
        "            print(f\"📈 Fetching {name} ({symbol})...\", end=\" \")\n",
        "            data = self.fetch_stock_data(symbol)\n",
        "            \n",
        "            if data is not None:\n",
        "                stock_data[symbol] = {\n",
        "                    'name': name,\n",
        "                    'data': data\n",
        "                }\n",
        "                print(\"✅\")\n",
        "            else:\n",
        "                failed_stocks.append(symbol)\n",
        "                print(\"❌\")\n",
        "        \n",
        "        print(f\"\\n✅ Successfully fetched: {len(stock_data)} stocks\")\n",
        "        if failed_stocks:\n",
        "            print(f\"❌ Failed to fetch: {failed_stocks}\")\n",
        "            \n",
        "        return stock_data\n",
        "\n",
        "# Initialize data fetcher\n",
        "data_fetcher = IndianStockDataFetcher(start_date='2022-01-01', end_date='2022-12-31')\n",
        "print(\"🔧 Indian Stock Data Fetcher initialized for 2022 analysis\")"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "id": "fetch_sample_data"
      },
      "outputs": [],
      "source": [
        "# Fetch sample data for initial analysis\n",
        "sample_stocks = {\n",
        "    'RELIANCE.NS': 'Reliance Industries',\n",
        "    'TCS.NS': 'Tata Consultancy Services', \n",
        "    'HDFCBANK.NS': 'HDFC Bank',\n",
        "    'INFY.NS': 'Infosys',\n",
        "    'ITC.NS': 'ITC Limited'\n",
        "}\n",
        "\n",
        "# Fetch the sample data\n",
        "sample_data = data_fetcher.fetch_multiple_stocks(sample_stocks)\n",
        "\n",
        "# Display basic info\n",
        "if sample_data:\n",
        "    print(\"\\n📊 Sample Data Overview:\")\n",
        "    for symbol, info in sample_data.items():\n",
        "        data = info['data']\n",
        "        print(f\"{info['name']}: {len(data)} trading days, Price range: ₹{data['low'].min():.2f} - ₹{data['high'].max():.2f}\")\n",
        "else:\n",
        "    print(\"❌ No sample data fetched successfully\")"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "technical_indicators"
      },
      "source": [
        "## 📊 Technical Indicators Implementation\n",
        "\n",
        "### Custom Technical Analysis Functions\n",
        "Implementing commonly used technical indicators for Indian stock market analysis."
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "id": "technical_indicators_class"
      },
      "outputs": [],
      "source": [
        "class TechnicalIndicators:\n",
        "    \"\"\"Custom technical indicators for Indian stock analysis\"\"\"\n",
        "    \n",
        "    @staticmethod\n",
        "    def sma(data, window):\n",
        "        \"\"\"Simple Moving Average\"\"\"\n",
        "        return data.rolling(window=window).mean()\n",
        "    \n",
        "    @staticmethod\n",
        "    def ema(data, window):\n",
        "        \"\"\"Exponential Moving Average\"\"\"\n",
        "        return data.ewm(span=window).mean()\n",
        "    \n",
        "    @staticmethod\n",
        "    def rsi(data, window=14):\n",
        "        \"\"\"Relative Strength Index\"\"\"\n",
        "        delta = data.diff()\n",
        "        gain = (delta.where(delta > 0, 0)).rolling(window=window).mean()\n",
        "        loss = (-delta.where(delta < 0, 0)).rolling(window=window).mean()\n",
        "        rs = gain / loss\n",
        "        rsi = 100 - (100 / (1 + rs))\n",
        "        return rsi\n",
        "    \n",
        "    @staticmethod\n",
        "    def macd(data, fast=12, slow=26, signal=9):\n",
        "        \"\"\"MACD Indicator\"\"\"\n",
        "        ema_fast = data.ewm(span=fast).mean()\n",
        "        ema_slow = data.ewm(span=slow).mean()\n",
        "        macd_line = ema_fast - ema_slow\n",
        "        signal_line = macd_line.ewm(span=signal).mean()\n",
        "        histogram = macd_line - signal_line\n",
        "        \n",
        "        return pd.DataFrame({\n",
        "            'MACD': macd_line,\n",
        "            'Signal': signal_line,\n",
        "            'Histogram': histogram\n",
        "        })\n",
        "    \n",
        "    @staticmethod\n",
        "    def bollinger_bands(data, window=20, num_std=2):\n",
        "        \"\"\"Bollinger Bands\"\"\"\n",
        "        sma = data.rolling(window=window).mean()\n",
        "        std = data.rolling(window=window).std()\n",
        "        \n",
        "        return pd.DataFrame({\n",
        "            'Middle': sma,\n",
        "            'Upper': sma + (std * num_std),\n",
        "            'Lower': sma - (std * num_std)\n",
        "        })\n",
        "    \n",
        "    @staticmethod\n",
        "    def stochastic(high, low, close, k_window=14, d_window=3):\n",
        "        \"\"\"Stochastic Oscillator\"\"\"\n",
        "        lowest_low = low.rolling(window=k_window).min()\n",
        "        highest_high = high.rolling(window=k_window).max()\n",
        "        \n",
        "        k_percent = 100 * ((close - lowest_low) / (highest_high - lowest_low))\n",
        "        d_percent = k_percent.rolling(window=d_window).mean()\n",
        "        \n",
        "        return pd.DataFrame({\n",
        "            '%K': k_percent,\n",
        "            '%D': d_percent\n",
        "        })\n",
        "    \n",
        "    @staticmethod\n",
        "    def atr(high, low, close, window=14):\n",
        "        \"\"\"Average True Range\"\"\"\n",
        "        high_low = high - low\n",
        "        high_close = np.abs(high - close.shift())\n",
        "        low_close = np.abs(low - close.shift())\n",
        "        \n",
        "        true_range = pd.concat([high_low, high_close, low_close], axis=1).max(axis=1)\n",
        "        return true_range.rolling(window=window).mean()\n",
        "\n",
        "# Initialize technical indicators\n",
        "ta = TechnicalIndicators()\n",
        "print(\"🔧 Technical Indicators module initialized\")\n",
        "print(\"Available indicators: SMA, EMA, RSI, MACD, Bollinger Bands, Stochastic, ATR\")"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "backtesting_framework"
      },
      "source": [
        "## 🔄 Backtesting Framework\n",
        "\n",
        "### Strategy Backtesting Engine\n",
        "Comprehensive backtesting framework for evaluating trading strategies on Indian stocks."
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "id": "backtesting_engine"
      },
      "outputs": [],
      "source": [
        "class BacktestingEngine:\n",
        "    \"\"\"Comprehensive backtesting engine for Indian stock strategies\"\"\"\n",
        "    \n",
        "    def __init__(self, initial_capital=100000, commission=0.001):\n",
        "        self.initial_capital = initial_capital\n",
        "        self.commission = commission  # 0.1% commission\n",
        "        self.results = {}\n",
        "        \n",
        "    def calculate_returns(self, data, signals):\n",
        "        \"\"\"Calculate strategy returns with commission\"\"\"\n",
        "        # Ensure signals and data are aligned\n",
        "        aligned_signals = signals.reindex(data.index).fillna(0)\n",
        "        \n",
        "        # Calculate daily returns\n",
        "        daily_returns = data['close'].pct_change()\n",
        "        \n",
        "        # Calculate position changes (for commission calculation)\n",
        "        position_changes = aligned_signals.diff().abs()\n",
        "        \n",
        "        # Calculate strategy returns (before commission)\n",
        "        strategy_returns = aligned_signals.shift(1) * daily_returns\n",
        "        \n",
        "        # Apply commission on position changes\n",
        "        commission_costs = position_changes * self.commission\n",
        "        \n",
        "        # Net strategy returns\n",
        "        net_strategy_returns = strategy_returns - commission_costs\n",
        "        \n",
        "        return net_strategy_returns.fillna(0)\n",
        "    \n",
        "    def calculate_metrics(self, returns, benchmark_returns=None):\n",
        "        \"\"\"Calculate comprehensive performance metrics\"\"\"\n",
        "        # Basic metrics\n",
        "        total_return = (1 + returns).prod() - 1\n",
        "        annualized_return = (1 + total_return) ** (252 / len(returns)) - 1\n",
        "        volatility = returns.std() * np.sqrt(252)\n",
        "        sharpe_ratio = annualized_return / volatility if volatility != 0 else 0\n",
        "        \n",
        "        # Drawdown calculations\n",
        "        cumulative_returns = (1 + returns).cumprod()\n",
        "        rolling_max = cumulative_returns.expanding().max()\n",
        "        drawdown = (cumulative_returns - rolling_max) / rolling_max\n",
        "        max_drawdown = drawdown.min()\n",
        "        \n",
        "        # Win rate\n",
        "        winning_days = (returns > 0).sum()\n",
        "        total_days = len(returns[returns != 0])\n",
        "        win_rate = winning_days / total_days if total_days > 0 else 0\n",
        "        \n",
        "        # Profit factor\n",
        "        gross_profit = returns[returns > 0].sum()\n",
        "        gross_loss = abs(returns[returns < 0].sum())\n",
        "        profit_factor = gross_profit / gross_loss if gross_loss != 0 else np.inf\n",
        "        \n",
        "        metrics = {\n",
        "            'Total Return': total_return,\n",
        "            'Annualized Return': annualized_return,\n",
        "            'Volatility': volatility,\n",
        "            'Sharpe Ratio': sharpe_ratio,\n",
        "            'Max Drawdown': max_drawdown,\n",
        "            'Win Rate': win_rate,\n",
        "            'Profit Factor': profit_factor,\n",
        "            'Total Trades': total_days\n",
        "        }\n",
        "        \n",
        "        # Benchmark comparison if provided\n",
        "        if benchmark_returns is not None:\n",
        "            benchmark_total_return = (1 + benchmark_returns).prod() - 1\n",
        "            alpha = total_return - benchmark_total_return\n",
        "            metrics['Alpha'] = alpha\n",
        "            metrics['Benchmark Return'] = benchmark_total_return\n",
        "        \n",
        "        return metrics\n",
        "    \n",
        "    def run_backtest(self, data, strategy_func, benchmark_data=None, **strategy_params):\n",
        "        \"\"\"Run backtest for a given strategy\"\"\"\n",
        "        try:\n",
        "            # Generate signals using the strategy function\n",
        "            signals = strategy_func(data, **strategy_params)\n",
        "            \n",
        "            # Calculate returns\n",
        "            strategy_returns = self.calculate_returns(data, signals)\n",
        "            \n",
        "            # Calculate benchmark returns if provided\n",
        "            benchmark_returns = None\n",
        "            if benchmark_data is not None:\n",
        "                benchmark_returns = benchmark_data['close'].pct_change().fillna(0)\n",
        "                # Align with strategy returns\n",
        "                benchmark_returns = benchmark_returns.reindex(strategy_returns.index).fillna(0)\n",
        "            \n",
        "            # Calculate metrics\n",
        "            metrics = self.calculate_metrics(strategy_returns, benchmark_returns)\n",
        "            \n",
        "            # Store results\n",
        "            results = {\n",
        "                'signals': signals,\n",
        "                'returns': strategy_returns,\n",
        "                'cumulative_returns': (1 + strategy_returns).cumprod(),\n",
        "                'metrics': metrics,\n",
        "                'benchmark_returns': benchmark_returns\n",
        "            }\n",
        "            \n",
        "            return results\n",
        "            \n",
        "        except Exception as e:\n",
        "            print(f\"❌ Backtest failed: {str(e)}\")\n",
        "            return None\n",
        "\n",
        "# Initialize backtesting engine\n",
        "backtest_engine = BacktestingEngine(initial_capital=100000, commission=0.001)\n",
        "print(\"🔧 Backtesting Engine initialized\")\n",
        "print(f\"💰 Initial Capital: ₹{backtest_engine.initial_capital:,}\")\n",
        "print(f\"💸 Commission: {backtest_engine.commission*100}%\")"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "trading_strategies"
      },
      "source": [
        "## 🎯 Trading Strategies Implementation\n",
        "\n",
        "### Strategy 1: Moving Average Crossover\n",
        "Classic moving average crossover strategy adapted for Indian markets."
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "id": "ma_crossover_strategy"
      },
      "outputs": [],
      "source": [
        "def moving_average_crossover_strategy(data, fast_window=10, slow_window=30, ma_type='SMA'):\n",
        "    \"\"\"Moving Average Crossover Strategy for Indian Stocks\"\"\"\n",
        "    \n",
        "    close_prices = data['close']\n",
        "    \n",
        "    # Calculate moving averages\n",
        "    if ma_type.upper() == 'SMA':\n",
        "        fast_ma = ta.sma(close_prices, fast_window)\n",
        "        slow_ma = ta.sma(close_prices, slow_window)\n",
        "    else:  # EMA\n",
        "        fast_ma = ta.ema(close_prices, fast_window)\n",
        "        slow_ma = ta.ema(close_prices, slow_window)\n",
        "    \n",
        "    # Generate signals\n",
        "    signals = pd.Series(0, index=data.index)\n",
        "    \n",
        "    # Buy signal: Fast MA crosses above Slow MA\n",
        "    bullish_crossover = (fast_ma > slow_ma) & (fast_ma.shift(1) <= slow_ma.shift(1))\n",
        "    \n",
        "    # Sell signal: Fast MA crosses below Slow MA  \n",
        "    bearish_crossover = (fast_ma < slow_ma) & (fast_ma.shift(1) >= slow_ma.shift(1))\n",
        "    \n",
        "    # Position sizing: 1 = Long, -1 = Short, 0 = No position\n",
        "    for i in range(1, len(signals)):\n",
        "        if bullish_crossover.iloc[i]:\n",
        "            signals.iloc[i:] = 1  # Enter long position\n",
        "        elif bearish_crossover.iloc[i]:\n",
        "            signals.iloc[i:] = 0  # Exit position (Indian markets - avoid shorting)\n",
        "        else:\n",
        "            signals.iloc[i] = signals.iloc[i-1]  # Hold previous position\n",
        "    \n",
        "    return signals\n",
        "\n",
        "print(\"📈 Moving Average Crossover Strategy implemented\")\n",
        "print(\"Features: SMA/EMA options, Long-only positions (suitable for Indian retail investors)\")"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "rsi_strategy"
      },
      "source": [
        "### Strategy 2: RSI Mean Reversion\n",
        "RSI-based mean reversion strategy optimized for Indian market volatility."
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "id": "rsi_mean_reversion"
      },
      "outputs": [],
      "source": [
        "def rsi_mean_reversion_strategy(data, rsi_window=14, oversold_threshold=30, overbought_threshold=70):\n",
        "    \"\"\"RSI Mean Reversion Strategy for Indian Stocks\"\"\"\n",
        "    \n",
        "    close_prices = data['close']\n",
        "    \n",
        "    # Calculate RSI\n",
        "    rsi = ta.rsi(close_prices, rsi_window)\n",
        "    \n",
        "    # Generate signals\n",
        "    signals = pd.Series(0, index=data.index)\n",
        "    \n",
        "    # Entry and exit conditions\n",
        "    oversold_condition = rsi < oversold_threshold\n",
        "    overbought_condition = rsi > overbought_threshold\n",
        "    neutral_condition = (rsi >= 45) & (rsi <= 55)  # Neutral zone for exits\n",
        "    \n",
        "    position = 0\n",
        "    for i in range(1, len(signals)):\n",
        "        if oversold_condition.iloc[i] and position == 0:\n",
        "            position = 1  # Enter long position when oversold\n",
        "        elif (overbought_condition.iloc[i] or neutral_condition.iloc[i]) and position == 1:\n",
        "            position = 0  # Exit when overbought or returning to neutral\n",
        "        \n",
        "        signals.iloc[i] = position\n",
        "    \n",
        "    return signals\n",
        "\n",
        "print(\"📊 RSI Mean Reversion Strategy implemented\")\n",
        "print(\"Features: Oversold/Overbought detection, Neutral zone exits\")"


       ]
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "bollinger_strategy"
      },
      "source": [
        "### Strategy 3: Bollinger Bands Breakout\n",
        "Bollinger Bands breakout strategy adapted for Indian market conditions."
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "id": "bollinger_breakout"
      },
      "outputs": [],
      "source": [
        "def bollinger_bands_strategy(data, bb_window=20, bb_std=2, volume_threshold=1.2):\n",
        "    \"\"\"Bollinger Bands Breakout Strategy for Indian Stocks\"\"\"\n",
        "    \n",
        "    close_prices = data['close']\n",
        "    volume = data['volume']\n",
        "    \n",
        "    # Calculate Bollinger Bands\n",
        "    bb = ta.bollinger_bands(close_prices, bb_window, bb_std)\n",
        "    \n",
        "    # Volume filter (Indian markets - volume confirmation important)\n",
        "    avg_volume = volume.rolling(window=20).mean()\n",
        "    high_volume = volume > (avg_volume * volume_threshold)\n",
        "    \n",
        "    # Generate signals\n",
        "    signals = pd.Series(0, index=data.index)\n",
        "    \n",
        "    # Breakout conditions\n",
        "    upper_breakout = (close_prices > bb['Upper']) & high_volume\n",
        "    lower_breakdown = (close_prices < bb['Lower']) & high_volume\n",
        "    middle_return = (close_prices > bb['Middle']) & (close_prices < bb['Upper'])\n",
        "    \n",
        "    position = 0\n",
        "    for i in range(1, len(signals)):\n",
        "        if upper_breakout.iloc[i] and position == 0:\n",
        "            position = 1  # Enter long on upper breakout with volume\n",
        "        elif (lower_breakdown.iloc[i] or middle_return.iloc[i]) and position == 1:\n",
        "            position = 0  # Exit on lower breakdown or return to middle\n",
        "        \n",
        "        signals.iloc[i] = position\n",
        "    \n",
        "    return signals\n",
        "\n",
        "print(\"📊 Bollinger Bands Breakout Strategy implemented\")\n",
        "print(\"Features: Volume confirmation, Upper breakout entries, Middle band exits\")"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "momentum_strategy"
      },
      "source": [
        "### Strategy 4: Momentum Strategy\n",
        "Multi-factor momentum strategy using price momentum and volume confirmation."
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "id": "momentum_strategy_impl"
      },
      "outputs": [],
      "source": [
        "def momentum_strategy(data, momentum_window=10, rsi_window=14, volume_window=20):\n",
        "    \"\"\"Multi-factor Momentum Strategy for Indian Stocks\"\"\"\n",
        "    \n",
        "    close_prices = data['close']\n",
        "    volume = data['volume']\n",
        "    \n",
        "    # Price momentum\n",
        "    price_momentum = (close_prices / close_prices.shift(momentum_window) - 1) * 100\n",
        "    \n",
        "    # RSI for momentum confirmation\n",
        "    rsi = ta.rsi(close_prices, rsi_window)\n",
        "    \n",
        "    # Volume trend\n",
        "    avg_volume = volume.rolling(window=volume_window).mean()\n",
        "    volume_ratio = volume / avg_volume\n",
        "    \n",
        "    # Generate signals\n",
        "    signals = pd.Series(0, index=data.index)\n",
        "    \n",
        "    # Momentum conditions (suitable for Indian markets)\n",
        "    strong_momentum = (price_momentum > 5) & (rsi > 60) & (volume_ratio > 1.5)\n",
        "    weak_momentum = (price_momentum < -3) | (rsi < 40) | (volume_ratio < 0.8)\n",
        "    \n",
        "    position = 0\n",
        "    for i in range(momentum_window, len(signals)):\n",
        "        if strong_momentum.iloc[i] and position == 0:\n",
        "            position = 1  # Enter long on strong momentum\n",
        "        elif weak_momentum.iloc[i] and position == 1:\n",
        "            position = 0  # Exit on weak momentum\n",
        "        \n",
        "        signals.iloc[i] = position\n",
        "    \n",
        "    return signals\n",
        "\n",
        "print(\"🚀 Momentum Strategy implemented\")\n",
        "print(\"Features: Price momentum, RSI confirmation, Volume trend analysis\")"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "strategy_testing"
      },
      "source": [
        "## 🧪 Strategy Testing & Results\n",
        "\n",
        "### Running Backtests on Sample Indian Stocks\n",
        "Testing all strategies on our sample Indian stocks with comprehensive analysis."
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "id": "run_backtests"
      },
      "outputs": [],
      "source": [
        "def run_comprehensive_backtests(stock_data, benchmark_symbol='^NSEI'):\n",
        "    \"\"\"Run all strategies on all stocks and compile results\"\"\"\n",
        "    \n",
        "    strategies = {\n",
        "        'MA_Crossover_10_30': lambda data: moving_average_crossover_strategy(data, 10, 30, 'SMA'),\n",
        "        'MA_Crossover_5_20': lambda data: moving_average_crossover_strategy(data, 5, 20, 'EMA'),\n",
        "        'RSI_Mean_Reversion': lambda data: rsi_mean_reversion_strategy(data, 14, 30, 70),\n",
        "        'RSI_Aggressive': lambda data: rsi_mean_reversion_strategy(data, 14, 25, 75),\n",
        "        'Bollinger_Breakout': lambda data: bollinger_bands_strategy(data, 20, 2, 1.2),\n",
        "        'Momentum_Strategy': lambda data: momentum_strategy(data, 10, 14, 20)\n",
        "    }\n",
        "    \n",
        "    # Fetch Nifty 50 as benchmark\n",
        "    print(f\"📈 Fetching benchmark data: Nifty 50...\")\n",
        "    benchmark_data = data_fetcher.fetch_stock_data(benchmark_symbol)\n",
        "    \n",
        "    all_results = {}\n",
        "    \n",
        "    print(f\"\\n🔄 Running backtests for {len(stock_data)} stocks and {len(strategies)} strategies...\")\n",
        "    print(\"=\" * 80)\n",
        "    \n",
        "    for symbol, stock_info in stock_data.items():\n",
        "        stock_name = stock_info['name']\n",
        "        data = stock_info['data']\n",
        "        \n",
        "        print(f\"\\n📊 Testing {stock_name} ({symbol})\")\n",
        "        print(\"-\" * 50)\n",
        "        \n",
        "        stock_results = {}\n",
        "        \n",
        "        for strategy_name, strategy_func in strategies.items():\n",
        "            print(f\"  🔄 {strategy_name}...\", end=\" \")\n",
        "            \n",
        "            try:\n",
        "                result = backtest_engine.run_backtest(\n",
        "                    data, \n",
        "                    strategy_func, \n",
        "                    benchmark_data\n",
        "                )\n",
        "                \n",
        "                if result:\n",
        "                    stock_results[strategy_name] = result\n",
        "                    \n",
        "                    # Quick metrics display\n",
        "                    total_return = result['metrics']['Total Return']\n",
        "                    sharpe_ratio = result['metrics']['Sharpe Ratio']\n",
        "                    max_drawdown = result['metrics']['Max Drawdown']\n",
        "                    \n",
        "                    print(f\"✅ Return: {total_return:.2%}, Sharpe: {sharpe_ratio:.2f}, DD: {max_drawdown:.2%}\")\n",
        "                else:\n",
        "                    print(\"❌ Failed\")\n",
        "                    \n",
        "            except Exception as e:\n",
        "                print(f\"❌ Error: {str(e)}\")\n",
        "        \n",
        "        all_results[symbol] = {\n",
        "            'name': stock_name,\n",
        "            'results': stock_results\n",
        "        }\n",
        "    \n",
        "    return all_results\n",
        "\n",
        "# Run comprehensive backtests\n",
        "if sample_data:\n",
        "    print(\"🚀 Starting comprehensive backtesting...\")\n",
        "    backtest_results = run_comprehensive_backtests(sample_data)\n",
        "    print(\"\\n✅ Backtesting completed!\")\n",
        "else:\n",
        "    print(\"❌ No sample data available for backtesting\")"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "results_analysis"
      },
      "source": [
        "## 📊 Results Analysis & Performance Comparison\n",
        "\n",
        "### Performance Summary Table\n",
        "Comprehensive analysis of all strategies across all tested stocks."
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "id": "performance_analysis"
      },
      "outputs": [],
      "source": [
        "def create_performance_summary(results):\n",
        "    \"\"\"Create comprehensive performance summary table\"\"\"\n",
        "    \n",
        "    summary_data = []\n",
        "    \n",
        "    for symbol, stock_info in results.items():\n",
        "        stock_name = stock_info['name']\n",
        "        \n",
        "        for strategy_name, result in stock_info['results'].items():\n",
        "            metrics = result['metrics']\n",
        "            \n",
        "            summary_data.append({\n",
        "                'Stock': stock_name,\n",
        "                'Symbol': symbol,\n",
        "                'Strategy': strategy_name,\n",
        "                'Total Return': metrics['Total Return'],\n",
        "                'Annualized Return': metrics['Annualized Return'],\n",
        "                'Volatility': metrics['Volatility'],\n",
        "                'Sharpe Ratio': metrics['Sharpe Ratio'],\n",
        "                'Max Drawdown': metrics['Max Drawdown'],\n",
        "                'Win Rate': metrics['Win Rate'],\n",
        "                'Profit Factor': metrics['Profit Factor'],\n",
        "                'Total Trades': metrics['Total Trades'],\n",
        "                'Alpha': metrics.get('Alpha', 0)\n",
        "            })\n",
        "    \n",
        "    df_summary = pd.DataFrame(summary_data)\n",
        "    return df_summary\n",
        "\n",
        "def analyze_best_strategies(summary_df):\n",
        "    \"\"\"Analyze and rank best performing strategies\"\"\"\n",
        "    \n",
        "    print(\"🏆 BEST PERFORMING STRATEGIES - 2022 Indian Market Analysis\")\n",
        "    print(\"=\" * 80)\n",
        "    \n",
        "    # Top strategies by Sharpe Ratio\n",
        "    top_sharpe = summary_df.nlargest(10, 'Sharpe Ratio')[['Stock', 'Strategy', 'Sharpe Ratio', 'Total Return', 'Max Drawdown']]\n",
        "    print(\"\\n📈 Top 10 Strategies by Sharpe Ratio:\")\n",
        "    print(top_sharpe.to_string(index=False, float_format='%.3f'))\n",
        "    \n",
        "    # Top strategies by Total Return\n",
        "    top_returns = summary_df.nlargest(10, 'Total Return')[['Stock', 'Strategy', 'Total Return', 'Sharpe Ratio', 'Max Drawdown']]\n",
        "    print(\"\\n💰 Top 10 Strategies by Total Return:\")\n",
        "    print(top_returns.to_string(index=False, float_format='%.3f'))\n",
        "    \n",
        "    # Strategy performance by category\n",
        "    strategy_avg = summary_df.groupby('Strategy').agg({\n",
        "        'Total Return': 'mean',\n",
        "        'Sharpe Ratio': 'mean',\n",
        "        'Max Drawdown': 'mean',\n",
        "        'Win Rate': 'mean'\n",
        "    }).round(3)\n",
        "    \n",
        "    print(\"\\n📊 Average Performance by Strategy Type:\")\n",
        "    print(strategy_avg.to_string())\n",
        "    \n",
        "    # Stock performance analysis\n",
        "    stock_avg = summary_df.groupby('Stock').agg({\n",
        "        'Total Return': 'mean',\n",
        "        'Sharpe Ratio': 'mean',\n",
        "        'Max Drawdown': 'mean',\n",
        "        'Alpha': 'mean'\n",
        "    }).round(3)\n",
        "    \n",
        "    print(\"\\n🏢 Average Performance by Stock:\")\n",
        "    print(stock_avg.to_string())\n",
        "    \n",
        "    return top_sharpe, top_returns, strategy_avg, stock_avg\n",
        "\n",
        "# Analyze results if backtesting was successful\n",
        "if 'backtest_results' in locals() and backtest_results:\n",
        "    performance_summary = create_performance_summary(backtest_results)\n",
        "    \n",
        "    print(f\"\\n📊 Performance Summary Created: {len(performance_summary)} strategy-stock combinations\")\n",
        "    \n",
        "    # Detailed analysis\n",
        "    top_sharpe, top_returns, strategy_avg, stock_avg = analyze_best_strategies(performance_summary)\n",
        "    \n",
        "else:\n",
        "    print(\"⚠️ No backtest results available for analysis\")"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "visualization"
      },
      "source": [
        "## 📈 Visualization & Reporting\n",
        "\n",
        "### Interactive Performance Charts\n",
        "Visual analysis of strategy performance with Indian market context."
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "id": "performance_visualization"
      },
      "outputs": [],
      "source": [
        "def create_performance_visualizations(results, summary_df):\n",
        "    \"\"\"Create comprehensive performance visualizations\"\"\"\n",
        "    \n",
        "    # 1. Strategy Performance Heatmap\n",
        "    fig, axes = plt.subplots(2, 2, figsize=(20, 16))\n",
        "    fig.suptitle('Indian Stock Market Strategy Performance Analysis - 2022', fontsize=16, fontweight='bold')\n",
        "    \n",
        "    # Heatmap of Sharpe Ratios\n",
        "    pivot_sharpe = summary_df.pivot(index='Stock', columns='Strategy', values='Sharpe Ratio')\n",
        "    sns.heatmap(pivot_sharpe, annot=True, cmap='RdYlGn', center=0, \n",
        "                ax=axes[0,0], fmt='.2f', cbar_kws={'label': 'Sharpe Ratio'})\n",
        "    axes[0,0].set_title('Sharpe Ratio by Stock & Strategy')\n",
        "    axes[0,0].tick_params(axis='x', rotation=45)\n",
        "    \n",
        "    # Heatmap of Total Returns\n",
        "    pivot_returns = summary_df.pivot(index='Stock', columns='Strategy', values='Total Return')\n",
        "    sns.heatmap(pivot_returns, annot=True, cmap='RdYlGn', center=0, \n",
        "                ax=axes[0,1], fmt='.2%', cbar_kws={'label': 'Total Return'})\n",
        "    axes[0,1].set_title('Total Return by Stock & Strategy')\n",
        "    axes[0,1].tick_params(axis='x', rotation=45)\n",
        "    \n",
        "    # Strategy Performance Box Plot\n",
        "    summary_df.boxplot(column='Sharpe Ratio', by='Strategy', ax=axes[1,0])\n",
        "    axes[1,0].set_title('Sharpe Ratio Distribution by Strategy')\n",
        "    axes[1,0].tick_params(axis='x', rotation=45)\n",
        "    \n",
        "    # Risk-Return Scatter Plot\n",
        "    for strategy in summary_df['Strategy'].unique():\n",
        "        strategy_data = summary_df[summary_df['Strategy'] == strategy]\n",
        "        axes[1,1].scatter(strategy_data['Volatility'], strategy_data['Annualized Return'], \n",
        "                         label=strategy, alpha=0.7, s=60)\n",
        "    \n",
        "    axes[1,1].set_xlabel('Volatility (Annual)')\n",
        "    axes[1,1].set_ylabel('Annualized Return')\n",
        "    axes[1,1].set_title('Risk-Return Profile by Strategy')\n",
        "    axes[1,1].legend(bbox_to_anchor=(1.05, 1), loc='upper left')\n",
        "    axes[1,1].grid(True, alpha=0.3)\n",
        "    \n",
        "    plt.tight_layout()\n",
        "    plt.show()\n",
        "    \n",
        "    # 2. Cumulative Returns Chart for Best Strategies\n",
        "    fig, ax = plt.subplots(figsize=(15, 8))\n",
        "    \n",
        "    # Get top 3 strategies by Sharpe ratio\n",
        "    top_strategies = summary_df.nlargest(3, 'Sharpe Ratio')\n",
        "    \n",
        "    for _, row in top_strategies.iterrows():\n",
        "        symbol = row['Symbol']\n",
        "        strategy = row['Strategy']\n",
        "        \n",
        "        if symbol in results and strategy in results[symbol]['results']:\n",
        "            cum_returns = results[symbol]['results'][strategy]['cumulative_returns']\n",
        "            ax.plot(cum_returns.index, cum_returns, \n",
        "                   label=f\"{row['Stock']} - {strategy}\", linewidth=2)\n",
        "    \n",
        "    # Add Nifty 50 benchmark if available\n",
        "    if 'benchmark_returns' in results[list(results.keys())[0]]['results'][list(results[list(results.keys())[0]]['results'].keys())[0]]:\n",
        "        benchmark = results[list(results.keys())[0]]['results'][list(results[list(results.keys())[0]]['results'].keys())[0]]['benchmark_returns']\n",
        "        if benchmark is not None:\n",
        "            benchmark_cum = (1 + benchmark).cumprod()\n",
        "            ax.plot(benchmark_cum.index, benchmark_cum, \n",
        "                   label='Nifty 50 Benchmark', linewidth=2, linestyle='--', color='black')\n",
        "    \n",
        "    ax.set_title('Cumulative Returns - Top Performing Strategies vs Nifty 50', fontsize=14, fontweight='bold')\n",
        "    ax.set_xlabel('Date')\n",
        "    ax.set_ylabel('Cumulative Returns')\n",
        "    ax.legend()\n",
        "    ax.grid(True, alpha=0.3)\n",
        "    \n",
        "    # Format y-axis as percentage\n",
        "    ax.yaxis.set_major_formatter(plt.FuncFormatter(lambda y, _: '{:.0%}'.format(y)))\n",
        "    \n",
        "    plt.tight_layout()\n",
        "    plt.show()\n",
        "    \n",
        "    print(\"📊 Performance visualizations created successfully!\")\n",
        "\n",
        "# Create visualizations if data is available\n",
        "if 'performance_summary' in locals() and not performance_summary.empty:\n",
        "    create_performance_visualizations(backtest_results, performance_summary)\n",
        "else:\n",
        "    print(\"⚠️ No performance data available for visualization\")"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "report_generation"
      },
      "source": [
        "## 📋 Final Report Generation\n",
        "\n",
        "### Comprehensive Research Report\n",
        "Generate detailed research report suitable for clients and fund managers."
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "id": "generate_report"
      },
      "outputs": [],
      "source": [
        "def generate_comprehensive_report(summary_df, strategy_avg, stock_avg):\n",
        "    \"\"\"Generate comprehensive research report for 2022 Indian market analysis\"\"\"\n",
        "    \n",
        "    report = f\"\"\"\n",
        "# 🇮🇳 INDIAN STOCK MARKET RESEARCH REPORT - 2022\n",
        "## Quantitative Strategy Backtesting Analysis\n",
        "\n",
        "**Report Date:** {pd.Timestamp.now().strftime('%Y-%m-%d %H:%M:%S')}\n",
        "**Analysis Period:** January 2022 - December 2022\n",
        "**Market Focus:** Indian Equity Markets (NSE)\n",
        "**Research Methodology:** Systematic backtesting of multiple quantitative strategies\n",
        "\n",
        "---\n",
        "\n",
        "## 📊 EXECUTIVE SUMMARY\n",
        "\n",
        "This report presents the results of comprehensive backtesting analysis conducted on {len(summary_df['Stock'].unique())} major Indian stocks using {len(summary_df['Strategy'].unique())} different quantitative trading strategies during 2022.\n",
        "\n",
        "### Key Findings:\n",
        "- **Total Strategy-Stock Combinations Tested:** {len(summary_df)}\n",
        "- **Best Performing Strategy:** {summary_df.loc[summary_df['Sharpe Ratio'].idxmax(), 'Strategy']}\n",
        "- **Highest Sharpe Ratio Achieved:** {summary_df['Sharpe Ratio'].max():.3f}\n",
        "- **Best Total Return:** {summary_df['Total Return'].max():.2%}\n",
        "- **Average Win Rate Across All Strategies:** {summary_df['Win Rate'].mean():.2%}\n",
        "\n",
        "---\n",
        "\n",
        "## 🎯 STRATEGY PERFORMANCE ANALYSIS\n",
        "\n",
        "### Average Performance by Strategy Type:\n",
        "\n",
        "\"\"\"\n",
        "    \n",
        "    # Add strategy performance table\n",
        "    report += \"```\\n\"\n",
        "    report += strategy_avg.to_string()\n",
        "    report += \"\\n```\\n\\n\"\n",
        "    \n",
        "    # Top performers\n",
        "    top_3_strategies = summary_df.nlargest(3, 'Sharpe Ratio')\n",
        "    \n",
        "    report += \"### 🏆 TOP 3 PERFORMING STRATEGIES:\\n\\n\"\n",
        "    \n",
        "    for i, (_, row) in enumerate(top_3_strategies.iterrows(), 1):\n",
        "        report += f\"\"\"\n",
        "**{i}. {row['Stock']} - {row['Strategy']}**\n",
        "- Total Return: {row['Total Return']:.2%}\n",
        "- Sharpe Ratio: {row['Sharpe Ratio']:.3f}\n",
        "- Max Drawdown: {row['Max Drawdown']:.2%}\n",
        "- Win Rate: {row['Win Rate']:.2%}\n",
        "- Alpha vs Nifty: {row['Alpha']:.2%}\n",
        "\"\"\"\n",
        "    \n",
        "    report += f\"\"\"\n",
        "\n",
        "---\n",
        "\n",
        "## 🏢 STOCK-SPECIFIC ANALYSIS\n",
        "\n",
        "### Average Performance by Stock:\n",
        "\n",
        "```\n",
        "{stock_avg.to_string()}\n",
        "```\n",
        "\n",
        "### Key Observations:\n",
        "\n",
        "1. **Sectoral Performance:**\n",
        "   - Banking stocks showed {('consistent' if stock_avg.loc[stock_avg.index.str.contains('Bank', na=False), 'Sharpe Ratio'].mean() > 0.5 else 'mixed')} performance\n",
        "   - IT stocks demonstrated {('strong' if 'TCS' in stock_avg.index and stock_avg.loc['TCS', 'Sharpe Ratio'] > 0.5 else 'moderate')} momentum characteristics\n",
        "   - Large-cap stocks showed {('lower' if stock_avg['Max Drawdown'].mean() < -0.15 else 'higher')} volatility than expected\n",
        "\n",
        "2. **Strategy Effectiveness:**\n",
        "   - Mean reversion strategies worked {('well' if strategy_avg.loc[strategy_avg.index.str.contains('RSI', na=False), 'Sharpe Ratio'].mean() > 0.3 else 'moderately')} in 2022 Indian markets\n",
        "   - Momentum strategies showed {('strong' if strategy_avg.loc[strategy_avg.index.str.contains('Momentum', na=False), 'Sharpe Ratio'].mean() > 0.3 else 'mixed')} results\n",
        "   - Moving average strategies provided {('consistent' if strategy_avg.loc[strategy_avg.index.str.contains('MA', na=False), 'Sharpe Ratio'].mean() > 0.2 else 'variable')} returns\n",
        "\n",
        "---\n",
        "\n",
        "## ⚠️ RISK ANALYSIS\n",
        "\n",
        "### Risk Metrics Summary:\n",
        "- **Average Maximum Drawdown:** {summary_df['Max Drawdown'].mean():.2%}\n",
        "- **Worst Drawdown Recorded:** {summary_df['Max Drawdown'].min():.2%}\n",
        "- **Average Volatility:** {summary_df['Volatility'].mean():.2%}\n",
        "- **Strategies with Positive Alpha:** {(summary_df['Alpha'] > 0).sum()}/{len(summary_df)}\n",
        "\n",
        "### Risk Management Observations:\n",
        "- Position sizing and risk management crucial for Indian markets\n",
        "- Volume confirmation important for breakout strategies\n",
        "- Sector rotation effects visible in 2022 data\n",
        "\n",
        "---\n",
        "\n",
        "## 💡 RECOMMENDATIONS FOR 2023\n",
        "\n",
        "Based on 2022 analysis:\n",
        "\n",
        "1. **Strategy Focus:**\n",
        "   - Prioritize strategies with Sharpe ratio > 0.5\n",
        "   - Combine momentum and mean reversion approaches\n",
        "   - Implement sector-specific parameters\n",
        "\n",
        "2. **Risk Management:**\n",
        "   - Maximum position size: 5% per stock\n",
        "   - Stop-loss at 8-10% for momentum strategies\n",
        "   - Volume confirmation mandatory for all entries\n",
        "\n",
        "3. **Portfolio Construction:**\n",
        "   - Diversify across sectors (Banking, IT, Pharma, Auto)\n",
        "   - Consider market cap allocation\n",
        "   - Regular rebalancing based on volatility changes\